# Compute Joint Exposure (Very High FHSZ ∩ Historical Fire)

This notebook computes joint exposure for publicly identifiable ≥115 kV transmission lines where two conditions co-occur:
1) The line intersects areas classified as Very High Fire Hazard Severity Zone (FHSZ), and
2) The line intersects the union of historical CAL FIRE FRAP fire perimeters.

Joint exposure is a screening-level indicator of where modeled hazard and observed historical fire occurrence overlap the same infrastructure geometry. It does not imply vulnerability, ignition likelihood, outages, or damage. All length calculations use EPSG:3310 and are reported in kilometers.

## Inputs (PostGIS)
- `tx_lines_3310`
- `hazard_fhsz_3310`
- `fire_perimeters_3310`

## Outputs (PostGIS)
- `tx_overlap_joint_by_line` (line_id, joint_km)

## Outputs (Files)
- `outputs/tables/tx_joint_exposure_summary.csv`
- `outputs/tables/tx_joint_exposure_by_line.csv`
- `outputs/tables/tx_joint_exposure_top25_lines.csv`

### Imports

In [1]:
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

### Define Paths

In [2]:
CWD = Path.cwd()
ROOT = CWD.parent if CWD.name.lower() == "notebooks" else CWD

OUTPUT_TABLES = ROOT / "outputs" / "tables"
OUTPUT_TABLES.mkdir(parents=True, exist_ok=True)

load_dotenv(ROOT / ".env")
print("ROOT:", ROOT)
print("OUTPUT_TABLES:", OUTPUT_TABLES)

ROOT: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure
OUTPUT_TABLES: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables


### Database Connection

In [3]:
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "wildfire_grid")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

if not DB_USER or not DB_PASSWORD:
    raise ValueError("Missing DB_USER or DB_PASSWORD in environment (.env).")

db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)

with engine.begin() as conn:
    conn.execute(text("SELECT 1;"))

print("DB:", DB_NAME, "@", DB_HOST, DB_PORT)

DB: wildfire_grid @ localhost 5432


## Prechecks

In [4]:
check_sql = """
SELECT
  to_regclass('public.tx_lines_3310') AS has_tx_lines_3310,
  to_regclass('public.hazard_fhsz_3310') AS has_hazard_fhsz_3310,
  to_regclass('public.fire_perimeters_3310') AS has_fire_perimeters_3310;
"""
pd.read_sql(check_sql, engine)

,has_tx_lines_3310,has_hazard_fhsz_3310,has_fire_perimeters_3310
0,tx_lines_3310,hazard_fhsz_3310,fire_perimeters_3310


## Cache “Very High Hazard” Geometry & Fire Union Geometry

In [5]:
sql_cache_joint_inputs = """
DROP TABLE IF EXISTS hazard_vh_3310;
CREATE TABLE hazard_vh_3310 AS
SELECT geom
FROM hazard_fhsz_3310
WHERE hazard_class = 'Very High';

CREATE INDEX IF NOT EXISTS idx_hazard_vh_3310_geom_gist ON hazard_vh_3310 USING GIST (geom);
ANALYZE hazard_vh_3310;

-- Single union geometry for all historical perimeters
DROP TABLE IF EXISTS fire_union_3310;
CREATE TABLE fire_union_3310 AS
SELECT ST_UnaryUnion(ST_Collect(geom)) AS geom
FROM fire_perimeters_3310;

CREATE INDEX IF NOT EXISTS idx_fire_union_3310_geom_gist ON fire_union_3310 USING GIST (geom);
ANALYZE fire_union_3310;
"""

with engine.begin() as conn:
    conn.execute(text(sql_cache_joint_inputs))

print("Created: hazard_vh_3310, fire_union_3310")

Created: hazard_vh_3310, fire_union_3310


## Compute Joint Overlap by Line

In [6]:
sql_joint_overlap = """
DROP TABLE IF EXISTS tx_overlap_joint_by_line;

CREATE TABLE tx_overlap_joint_by_line AS
WITH vh_clip AS (
  SELECT
    t.line_id,
    ST_Intersection(t.geom, vh.geom) AS geom_vh
  FROM tx_lines_3310 t
  JOIN hazard_vh_3310 vh
    ON ST_Intersects(t.geom, vh.geom)
),
vh_lines AS (
  SELECT
    line_id,
    ST_CollectionExtract(geom_vh, 2) AS geom_vh_line
  FROM vh_clip
  WHERE geom_vh IS NOT NULL AND NOT ST_IsEmpty(geom_vh)
),
joint_clip AS (
  SELECT
    v.line_id,
    ST_Intersection(v.geom_vh_line, f.geom) AS geom_joint
  FROM vh_lines v
  CROSS JOIN fire_union_3310 f
  WHERE ST_Intersects(v.geom_vh_line, f.geom)
),
joint_lines AS (
  SELECT
    line_id,
    ST_CollectionExtract(geom_joint, 2) AS geom_joint_line
  FROM joint_clip
  WHERE geom_joint IS NOT NULL AND NOT ST_IsEmpty(geom_joint)
)
SELECT
  line_id,
  (ST_Length(geom_joint_line) / 1000.0) AS joint_km
FROM joint_lines
WHERE geom_joint_line IS NOT NULL
  AND NOT ST_IsEmpty(geom_joint_line);

CREATE INDEX IF NOT EXISTS idx_tx_overlap_joint_line ON tx_overlap_joint_by_line (line_id);
ANALYZE tx_overlap_joint_by_line;
"""

with engine.begin() as conn:
    conn.execute(text(sql_joint_overlap))

print("Created: tx_overlap_joint_by_line")

Created: tx_overlap_joint_by_line


## Summary Table

In [7]:
df_summary = pd.read_sql(
    """
    SELECT
      SUM(joint_km) AS total_joint_km,
      COUNT(*) AS n_lines_with_joint_exposure
    FROM tx_overlap_joint_by_line;
    """,
    engine
)
df_summary

,total_joint_km,n_lines_with_joint_exposure
0,57.041651,78


## Export CSVs

In [8]:
df_summary.to_csv(OUTPUT_TABLES / "tx_joint_exposure_summary.csv", index=False)

df_joint_by_line = pd.read_sql(
    """
    SELECT line_id, joint_km
    FROM tx_overlap_joint_by_line
    ORDER BY joint_km DESC;
    """,
    engine
)
df_joint_by_line.to_csv(OUTPUT_TABLES / "tx_joint_exposure_by_line.csv", index=False)

df_top25 = pd.read_sql(
    """
    SELECT
      t.line_id,
      t.owner,
      t.voltage_kv,
      j.joint_km
    FROM tx_overlap_joint_by_line j
    JOIN tx_lines_3310 t USING (line_id)
    ORDER BY j.joint_km DESC
    LIMIT 25;
    """,
    engine
)
df_top25.to_csv(OUTPUT_TABLES / "tx_joint_exposure_top25_lines.csv", index=False)

print("Wrote:")
print("-", OUTPUT_TABLES / "tx_joint_exposure_summary.csv")
print("-", OUTPUT_TABLES / "tx_joint_exposure_by_line.csv")
print("-", OUTPUT_TABLES / "tx_joint_exposure_top25_lines.csv")

Wrote:
- C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables\tx_joint_exposure_summary.csv
- C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables\tx_joint_exposure_by_line.csv
- C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables\tx_joint_exposure_top25_lines.csv
